In [14]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score,roc_auc_score
from lightgbm import LGBMClassifier
import pandas as pd
import numpy as np

final12 = pd.read_csv('final_df12_new3.csv', index_col=0)
final13 = pd.read_csv('final13_new.csv', index_col=0)

## 1-2분기 학습, 3분기 예측

In [11]:
final12

,CUSTNO,GENDER,AGE,가공_avg,가구/인테_avg,일상_avg,스포츠_avg,패션잡화_avg,교육/문화_avg,기타_avg,가전_avg,의류_avg,신선_avg,의약품_avg,사치품_avg,avg_grade,GBTIME,target_grade,R_grade
0,1,0.0,6.0,4.0,8.0,7.5,4.5,1.5,0.0,4.0,4.0,5.0,7.0,0.0,3.0,6.0,3.0,0.0,5.0
1,2,0.0,6.0,4.0,6.0,7.0,9.0,1.0,0.0,0.0,8.0,5.0,7.0,0.0,3.0,3.0,3.0,1.0,5.0
2,3,0.0,6.0,1.0,0.0,2.5,0.0,0.0,1.0,0.0,2.0,2.5,1.0,0.0,0.0,1.0,3.0,0.0,4.0
3,4,1.0,6.0,4.0,4.5,4.0,7.0,1.5,0.5,0.0,0.0,5.0,5.5,0.0,0.0,2.0,3.0,0.0,5.0
4,5,0.0,6.0,4.0,6.0,6.0,5.0,3.0,0.0,0.0,4.0,5.0,7.0,0.0,3.0,2.0,3.0,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18532,19376,0.0,2.0,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,4.0,0.0,4.0
18533,19377,0.0,2.0,0.5,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,4.0,3.0,0.0,4.0
18534,19378,1.0,4.0,2.0,2.0,3.5,0.0,0.0,0.0,0.0,0.0,2.5,3.5,0.0,0.0,4.0,4.0,1.0,5.0
18535,19379,1.0,2.0,0.5,0.0,2.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,4.0,4.0,0.0,5.0


In [17]:
from sklearn.model_selection import GridSearchCV

x_features = final12.drop(['CUSTNO','target_grade'], axis=1, inplace=False)
y_target = final12['target_grade']

x_train,x_test,y_train,y_test = train_test_split(x_features, y_target, test_size=0.2, random_state=0)

LGBM_clf = LGBMClassifier(n_estimators=200)

param = {'num_leaves': [32, 64, 120],
          'max_depth':[128, 160, 200],
          'min_child_samples':[60, 100],
          'subsample':[0.8, 1,0.4]}
gridcv = GridSearchCV(LGBM_clf, param_grid=param)
gridcv.fit(x_train,y_train, early_stopping_rounds=30, eval_metric='accuracy',eval_set=[(x_train,y_train),(x_test,y_test)])
print(f'최적 파라미터 : {gridcv.best_params_}')
lgbm_roc_score = roc_auc_score(y_test,gridcv.predict_proba(x_test)[:,1], average='macro')
lgbm_accuracy = roc_auc_score(y_test,gridcv.predict(x_test), average='macro')
print(f'ROC_AUC : {round(lgbm_roc_score,4)}')
print(f'Accuracy : {round(lgbm_accuracy,4)}')

[1]	valid_0's binary_logloss: 0.528217	valid_1's binary_logloss: 0.528312
[2]	valid_0's binary_logloss: 0.513432	valid_1's binary_logloss: 0.515326
[3]	valid_0's binary_logloss: 0.501987	valid_1's binary_logloss: 0.505514
[4]	valid_0's binary_logloss: 0.492853	valid_1's binary_logloss: 0.497748
[5]	valid_0's binary_logloss: 0.485325	valid_1's binary_logloss: 0.491457
[6]	valid_0's binary_logloss: 0.479108	valid_1's binary_logloss: 0.486513
[7]	valid_0's binary_logloss: 0.474029	valid_1's binary_logloss: 0.482556
[8]	valid_0's binary_logloss: 0.469643	valid_1's binary_logloss: 0.479247
[9]	valid_0's binary_logloss: 0.466034	valid_1's binary_logloss: 0.476535
[10]	valid_0's binary_logloss: 0.462787	valid_1's binary_logloss: 0.474153
[11]	valid_0's binary_logloss: 0.460047	valid_1's binary_logloss: 0.4726
[12]	valid_0's binary_logloss: 0.457681	valid_1's binary_logloss: 0.471308
[13]	valid_0's binary_logloss: 0.455664	valid_1's binary_logloss: 0.470091
[14]	valid_0's binary_logloss: 0.453

[1]	valid_0's binary_logloss: 0.527921	valid_1's binary_logloss: 0.528204
[2]	valid_0's binary_logloss: 0.513036	valid_1's binary_logloss: 0.515013
[3]	valid_0's binary_logloss: 0.501358	valid_1's binary_logloss: 0.504942
[4]	valid_0's binary_logloss: 0.492173	valid_1's binary_logloss: 0.496963
[5]	valid_0's binary_logloss: 0.484663	valid_1's binary_logloss: 0.490965
[6]	valid_0's binary_logloss: 0.47854	valid_1's binary_logloss: 0.48591
[7]	valid_0's binary_logloss: 0.473385	valid_1's binary_logloss: 0.481732
[8]	valid_0's binary_logloss: 0.469124	valid_1's binary_logloss: 0.478684
[9]	valid_0's binary_logloss: 0.465442	valid_1's binary_logloss: 0.475968
[10]	valid_0's binary_logloss: 0.462266	valid_1's binary_logloss: 0.473589
[11]	valid_0's binary_logloss: 0.459586	valid_1's binary_logloss: 0.471766
[12]	valid_0's binary_logloss: 0.457251	valid_1's binary_logloss: 0.47027
[13]	valid_0's binary_logloss: 0.455264	valid_1's binary_logloss: 0.469067
[14]	valid_0's binary_logloss: 0.4536

[53]	valid_0's binary_logloss: 0.42578	valid_1's binary_logloss: 0.467431
[54]	valid_0's binary_logloss: 0.425281	valid_1's binary_logloss: 0.467432
[55]	valid_0's binary_logloss: 0.424712	valid_1's binary_logloss: 0.467155
[56]	valid_0's binary_logloss: 0.424298	valid_1's binary_logloss: 0.467465
[57]	valid_0's binary_logloss: 0.423804	valid_1's binary_logloss: 0.467576
[58]	valid_0's binary_logloss: 0.423301	valid_1's binary_logloss: 0.467607
[59]	valid_0's binary_logloss: 0.422933	valid_1's binary_logloss: 0.467628
[60]	valid_0's binary_logloss: 0.422313	valid_1's binary_logloss: 0.467694
[61]	valid_0's binary_logloss: 0.421823	valid_1's binary_logloss: 0.46782
[1]	valid_0's binary_logloss: 0.527846	valid_1's binary_logloss: 0.52792
[2]	valid_0's binary_logloss: 0.512831	valid_1's binary_logloss: 0.514595
[3]	valid_0's binary_logloss: 0.501228	valid_1's binary_logloss: 0.504527
[4]	valid_0's binary_logloss: 0.491978	valid_1's binary_logloss: 0.496563
[5]	valid_0's binary_logloss: 0.

[19]	valid_0's binary_logloss: 0.447155	valid_1's binary_logloss: 0.465566
[20]	valid_0's binary_logloss: 0.446123	valid_1's binary_logloss: 0.465293
[21]	valid_0's binary_logloss: 0.445219	valid_1's binary_logloss: 0.465179
[22]	valid_0's binary_logloss: 0.444253	valid_1's binary_logloss: 0.464975
[23]	valid_0's binary_logloss: 0.443342	valid_1's binary_logloss: 0.464747
[24]	valid_0's binary_logloss: 0.442501	valid_1's binary_logloss: 0.464772
[25]	valid_0's binary_logloss: 0.441679	valid_1's binary_logloss: 0.464754
[26]	valid_0's binary_logloss: 0.440801	valid_1's binary_logloss: 0.464764
[27]	valid_0's binary_logloss: 0.440105	valid_1's binary_logloss: 0.464639
[28]	valid_0's binary_logloss: 0.439464	valid_1's binary_logloss: 0.464534
[29]	valid_0's binary_logloss: 0.438698	valid_1's binary_logloss: 0.464557
[30]	valid_0's binary_logloss: 0.438219	valid_1's binary_logloss: 0.4646
[31]	valid_0's binary_logloss: 0.437627	valid_1's binary_logloss: 0.464407
[32]	valid_0's binary_loglo

[61]	valid_0's binary_logloss: 0.421823	valid_1's binary_logloss: 0.46782
[1]	valid_0's binary_logloss: 0.527846	valid_1's binary_logloss: 0.52792
[2]	valid_0's binary_logloss: 0.512831	valid_1's binary_logloss: 0.514595
[3]	valid_0's binary_logloss: 0.501228	valid_1's binary_logloss: 0.504527
[4]	valid_0's binary_logloss: 0.491978	valid_1's binary_logloss: 0.496563
[5]	valid_0's binary_logloss: 0.484487	valid_1's binary_logloss: 0.490411
[6]	valid_0's binary_logloss: 0.478213	valid_1's binary_logloss: 0.485182
[7]	valid_0's binary_logloss: 0.473134	valid_1's binary_logloss: 0.480961
[8]	valid_0's binary_logloss: 0.468889	valid_1's binary_logloss: 0.477719
[9]	valid_0's binary_logloss: 0.465156	valid_1's binary_logloss: 0.474599
[10]	valid_0's binary_logloss: 0.462123	valid_1's binary_logloss: 0.4724
[11]	valid_0's binary_logloss: 0.459359	valid_1's binary_logloss: 0.470789
[12]	valid_0's binary_logloss: 0.45701	valid_1's binary_logloss: 0.469229
[13]	valid_0's binary_logloss: 0.454919

...

[41]	valid_0's binary_logloss: 0.410787	valid_1's binary_logloss: 0.471516
[42]	valid_0's binary_logloss: 0.4098	valid_1's binary_logloss: 0.471699
[43]	valid_0's binary_logloss: 0.408828	valid_1's binary_logloss: 0.472382
[44]	valid_0's binary_logloss: 0.407783	valid_1's binary_logloss: 0.472676
[45]	valid_0's binary_logloss: 0.406828	valid_1's binary_logloss: 0.472794
[46]	valid_0's binary_logloss: 0.406004	valid_1's binary_logloss: 0.47315
[47]	valid_0's binary_logloss: 0.404796	valid_1's binary_logloss: 0.473315
[48]	valid_0's binary_logloss: 0.403807	valid_1's binary_logloss: 0.473535
[49]	valid_0's binary_logloss: 0.402947	valid_1's binary_logloss: 0.473958
[50]	valid_0's binary_logloss: 0.402014	valid_1's binary_logloss: 0.474284
[51]	valid_0's binary_logloss: 0.400997	valid_1's binary_logloss: 0.474664
[1]	valid_0's binary_logloss: 0.5274	valid_1's binary_logloss: 0.527855
[2]	valid_0's binary_logloss: 0.511923	valid_1's binary_logloss: 0.514718
[3]	valid_0's binary_logloss: 0.

[22]	valid_0's binary_logloss: 0.432878	valid_1's binary_logloss: 0.468612
[23]	valid_0's binary_logloss: 0.431553	valid_1's binary_logloss: 0.468775
[24]	valid_0's binary_logloss: 0.430454	valid_1's binary_logloss: 0.469115
[25]	valid_0's binary_logloss: 0.429165	valid_1's binary_logloss: 0.469267
[26]	valid_0's binary_logloss: 0.428013	valid_1's binary_logloss: 0.469401
[27]	valid_0's binary_logloss: 0.426829	valid_1's binary_logloss: 0.469752
[28]	valid_0's binary_logloss: 0.425598	valid_1's binary_logloss: 0.469916
[29]	valid_0's binary_logloss: 0.424242	valid_1's binary_logloss: 0.470179
[30]	valid_0's binary_logloss: 0.422993	valid_1's binary_logloss: 0.470358
[31]	valid_0's binary_logloss: 0.421681	valid_1's binary_logloss: 0.470596
[32]	valid_0's binary_logloss: 0.420601	valid_1's binary_logloss: 0.471018
[33]	valid_0's binary_logloss: 0.419199	valid_1's binary_logloss: 0.471088
[34]	valid_0's binary_logloss: 0.418081	valid_1's binary_logloss: 0.47135
[35]	valid_0's binary_logl

[4]	valid_0's binary_logloss: 0.489884	valid_1's binary_logloss: 0.496539
[5]	valid_0's binary_logloss: 0.481761	valid_1's binary_logloss: 0.490333
[6]	valid_0's binary_logloss: 0.475109	valid_1's binary_logloss: 0.485287
[7]	valid_0's binary_logloss: 0.469441	valid_1's binary_logloss: 0.481414
[8]	valid_0's binary_logloss: 0.464553	valid_1's binary_logloss: 0.47857
[9]	valid_0's binary_logloss: 0.460513	valid_1's binary_logloss: 0.47627
[10]	valid_0's binary_logloss: 0.45688	valid_1's binary_logloss: 0.474256
[11]	valid_0's binary_logloss: 0.4537	valid_1's binary_logloss: 0.472768
[12]	valid_0's binary_logloss: 0.450895	valid_1's binary_logloss: 0.4719
[13]	valid_0's binary_logloss: 0.448482	valid_1's binary_logloss: 0.471164
[14]	valid_0's binary_logloss: 0.446318	valid_1's binary_logloss: 0.470426
[15]	valid_0's binary_logloss: 0.444454	valid_1's binary_logloss: 0.470149
[16]	valid_0's binary_logloss: 0.442676	valid_1's binary_logloss: 0.469423
[17]	valid_0's binary_logloss: 0.44106

[1]	valid_0's binary_logloss: 0.527068	valid_1's binary_logloss: 0.528021
[2]	valid_0's binary_logloss: 0.51152	valid_1's binary_logloss: 0.514742
[3]	valid_0's binary_logloss: 0.499289	valid_1's binary_logloss: 0.504928
[4]	valid_0's binary_logloss: 0.489446	valid_1's binary_logloss: 0.497033
[5]	valid_0's binary_logloss: 0.481268	valid_1's binary_logloss: 0.490763
[6]	valid_0's binary_logloss: 0.474546	valid_1's binary_logloss: 0.486102
[7]	valid_0's binary_logloss: 0.468851	valid_1's binary_logloss: 0.482097
[8]	valid_0's binary_logloss: 0.464129	valid_1's binary_logloss: 0.479039
[9]	valid_0's binary_logloss: 0.460113	valid_1's binary_logloss: 0.476443
[10]	valid_0's binary_logloss: 0.456432	valid_1's binary_logloss: 0.474624
[11]	valid_0's binary_logloss: 0.453378	valid_1's binary_logloss: 0.472621
[12]	valid_0's binary_logloss: 0.450459	valid_1's binary_logloss: 0.471101
[13]	valid_0's binary_logloss: 0.447674	valid_1's binary_logloss: 0.469788
[14]	valid_0's binary_logloss: 0.44

[1]	training's binary_logloss: 0.52795	valid_1's binary_logloss: 0.527889
[2]	training's binary_logloss: 0.512773	valid_1's binary_logloss: 0.514767
[3]	training's binary_logloss: 0.501033	valid_1's binary_logloss: 0.504814
[4]	training's binary_logloss: 0.49182	valid_1's binary_logloss: 0.496899
[5]	training's binary_logloss: 0.484314	valid_1's binary_logloss: 0.490827
[6]	training's binary_logloss: 0.478074	valid_1's binary_logloss: 0.485757
[7]	training's binary_logloss: 0.472833	valid_1's binary_logloss: 0.482021
[8]	training's binary_logloss: 0.468525	valid_1's binary_logloss: 0.478808
[9]	training's binary_logloss: 0.464785	valid_1's binary_logloss: 0.476126
[10]	training's binary_logloss: 0.461624	valid_1's binary_logloss: 0.473963
[11]	training's binary_logloss: 0.458881	valid_1's binary_logloss: 0.472149
[12]	training's binary_logloss: 0.456467	valid_1's binary_logloss: 0.470732
[13]	training's binary_logloss: 0.454379	valid_1's binary_logloss: 0.469674
[14]	training's binary_

In [ ]:
!pip install lazypredict

In [15]:
from lazypredict.Supervised import LazyClassifier, LazyRegressor

x_features = final12.drop(['CUSTNO','target_grade'], axis=1, inplace=False)
y_target = final12['target_grade']

x_train, x_test, y_train, y_test = train_test_split(x_features, y_target, test_size = 0.2, random_state=156)

clf = LazyClassifier(verbose=0, predictions=True)
# clf2 = LazyRegressor(verbose=0, predictions=True)

# models, predictions = clf2.fit(x_train, x_test, y_train, y_test)
models, predictions = clf.fit(x_train, x_test, y_train, y_test)
models

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:53<00:00,  1.85s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
AdaBoostClassifier,0.82,0.66,0.66,0.80,0.54
LGBMClassifier,0.82,0.66,0.66,0.79,0.21
RandomForestClassifier,0.81,0.65,0.65,0.79,1.61
XGBClassifier,0.81,0.65,0.65,0.79,0.58
ExtraTreesClassifier,0.80,0.64,0.64,0.78,1.63
QuadraticDiscriminantAnalysis,0.76,0.63,0.63,0.75,0.08
BaggingClassifier,0.79,0.63,0.63,0.77,0.47
SVC,0.79,0.61,0.61,0.76,12.79
NearestCentroid,0.63,0.61,0.61,0.66,0.03


## 1-3분기 학습, 4분기 예측

In [12]:
final13

,CUSTNO,GENDER,AGE,가공_avg,가구/인테_avg,일상_avg,스포츠_avg,패션잡화_avg,교육/문화_avg,기타_avg,가전_avg,의류_avg,신선_avg,의약품_avg,사치품_avg,avg_grade,GBTIME,target_grade,R_grade
0,1,0,6,4.0,8.0,7.0,5.0,3.0,0.0,3.0,5.0,5.0,7.0,0.0,4.0,3.0,3.0,0.0,4.0
1,2,0,6,4.0,7.0,7.0,9.0,1.0,0.0,0.0,8.0,5.0,7.0,0.0,4.0,6.0,3.0,1.0,4.0
2,3,0,6,1.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,1.0,3.0,0.0,4.0
3,4,1,6,4.0,4.0,4.0,6.0,1.0,0.0,0.0,0.0,5.0,6.0,1.0,0.0,2.0,3.0,0.0,5.0
4,5,0,6,4.0,4.0,7.0,3.0,2.0,0.0,0.0,3.0,6.0,7.0,0.0,2.0,5.0,3.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19378,19379,1,2,1.0,0.0,3.0,0.0,3.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,4.0,4.0,0.0,5.0
19379,19380,1,2,1.0,0.0,3.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,5.0
19380,19381,1,4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,3.0,0.0,4.0
19381,19382,1,4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,3.0,0.0,5.0


In [18]:
from sklearn.model_selection import GridSearchCV

x_features = final13.drop(['CUSTNO','target_grade'], axis=1, inplace=False)
y_target = final13['target_grade']

x_train,x_test,y_train,y_test = train_test_split(x_features, y_target, test_size=0.2, random_state=0)

LGBM_clf = LGBMClassifier(n_estimators=200)

param = {'num_leaves': [32, 64, 120],
          'max_depth':[128, 160, 200],
          'min_child_samples':[60, 100],
          'subsample':[0.8, 1,0.4]}
gridcv = GridSearchCV(LGBM_clf, param_grid=param)
gridcv.fit(x_train,y_train, early_stopping_rounds=30, eval_metric='accuracy',eval_set=[(x_train,y_train),(x_test,y_test)])
print(f'최적 파라미터 : {gridcv.best_params_}')
lgbm_roc_score = roc_auc_score(y_test,gridcv.predict_proba(x_test)[:,1], average='macro')
lgbm_accuracy = roc_auc_score(y_test,gridcv.predict(x_test), average='macro')
print(f'ROC_AUC : {round(lgbm_roc_score,4)}')
print(f'Accuracy : {round(lgbm_accuracy,4)}')

[1]	valid_0's binary_logloss: 0.623201	valid_1's binary_logloss: 0.633596
[2]	valid_0's binary_logloss: 0.610592	valid_1's binary_logloss: 0.623596
[3]	valid_0's binary_logloss: 0.600269	valid_1's binary_logloss: 0.615573
[4]	valid_0's binary_logloss: 0.591713	valid_1's binary_logloss: 0.60901
[5]	valid_0's binary_logloss: 0.58442	valid_1's binary_logloss: 0.603597
[6]	valid_0's binary_logloss: 0.57848	valid_1's binary_logloss: 0.598942
[7]	valid_0's binary_logloss: 0.573186	valid_1's binary_logloss: 0.595368
[8]	valid_0's binary_logloss: 0.568723	valid_1's binary_logloss: 0.592533
[9]	valid_0's binary_logloss: 0.564908	valid_1's binary_logloss: 0.590454
[10]	valid_0's binary_logloss: 0.561782	valid_1's binary_logloss: 0.588422
[11]	valid_0's binary_logloss: 0.558941	valid_1's binary_logloss: 0.586723
[12]	valid_0's binary_logloss: 0.556497	valid_1's binary_logloss: 0.585544
[13]	valid_0's binary_logloss: 0.554298	valid_1's binary_logloss: 0.584415
[14]	valid_0's binary_logloss: 0.5523

[1]	valid_0's binary_logloss: 0.622975	valid_1's binary_logloss: 0.633067
[2]	valid_0's binary_logloss: 0.610157	valid_1's binary_logloss: 0.622792
[3]	valid_0's binary_logloss: 0.599687	valid_1's binary_logloss: 0.614685
[4]	valid_0's binary_logloss: 0.590926	valid_1's binary_logloss: 0.607762
[5]	valid_0's binary_logloss: 0.583809	valid_1's binary_logloss: 0.602442
[6]	valid_0's binary_logloss: 0.57758	valid_1's binary_logloss: 0.597915
[7]	valid_0's binary_logloss: 0.572339	valid_1's binary_logloss: 0.594061
[8]	valid_0's binary_logloss: 0.567953	valid_1's binary_logloss: 0.591265
[9]	valid_0's binary_logloss: 0.564355	valid_1's binary_logloss: 0.588894
[10]	valid_0's binary_logloss: 0.561062	valid_1's binary_logloss: 0.587305
[11]	valid_0's binary_logloss: 0.558294	valid_1's binary_logloss: 0.58568
[12]	valid_0's binary_logloss: 0.555739	valid_1's binary_logloss: 0.58422
[13]	valid_0's binary_logloss: 0.553575	valid_1's binary_logloss: 0.582928
[14]	valid_0's binary_logloss: 0.5517

[38]	valid_0's binary_logloss: 0.531156	valid_1's binary_logloss: 0.581276
[39]	valid_0's binary_logloss: 0.530415	valid_1's binary_logloss: 0.581473
[40]	valid_0's binary_logloss: 0.529848	valid_1's binary_logloss: 0.581396
[41]	valid_0's binary_logloss: 0.529166	valid_1's binary_logloss: 0.581529
[42]	valid_0's binary_logloss: 0.528617	valid_1's binary_logloss: 0.581529
[43]	valid_0's binary_logloss: 0.528047	valid_1's binary_logloss: 0.581513
[44]	valid_0's binary_logloss: 0.527635	valid_1's binary_logloss: 0.581502
[45]	valid_0's binary_logloss: 0.527206	valid_1's binary_logloss: 0.581555
[46]	valid_0's binary_logloss: 0.526728	valid_1's binary_logloss: 0.581752
[47]	valid_0's binary_logloss: 0.526288	valid_1's binary_logloss: 0.581673
[48]	valid_0's binary_logloss: 0.52563	valid_1's binary_logloss: 0.581807
[49]	valid_0's binary_logloss: 0.524951	valid_1's binary_logloss: 0.58165
[50]	valid_0's binary_logloss: 0.524373	valid_1's binary_logloss: 0.581812
[51]	valid_0's binary_loglo

[59]	valid_0's binary_logloss: 0.517982	valid_1's binary_logloss: 0.581571
[60]	valid_0's binary_logloss: 0.517571	valid_1's binary_logloss: 0.581658
[61]	valid_0's binary_logloss: 0.517218	valid_1's binary_logloss: 0.581921
[62]	valid_0's binary_logloss: 0.516657	valid_1's binary_logloss: 0.581918
[63]	valid_0's binary_logloss: 0.516127	valid_1's binary_logloss: 0.581977
[1]	valid_0's binary_logloss: 0.622975	valid_1's binary_logloss: 0.633067
[2]	valid_0's binary_logloss: 0.610157	valid_1's binary_logloss: 0.622792
[3]	valid_0's binary_logloss: 0.599687	valid_1's binary_logloss: 0.614685
[4]	valid_0's binary_logloss: 0.590926	valid_1's binary_logloss: 0.607762
[5]	valid_0's binary_logloss: 0.583809	valid_1's binary_logloss: 0.602442
[6]	valid_0's binary_logloss: 0.57758	valid_1's binary_logloss: 0.597915
[7]	valid_0's binary_logloss: 0.572339	valid_1's binary_logloss: 0.594061
[8]	valid_0's binary_logloss: 0.567953	valid_1's binary_logloss: 0.591265
[9]	valid_0's binary_logloss: 0.56

[8]	valid_0's binary_logloss: 0.568723	valid_1's binary_logloss: 0.592533
[9]	valid_0's binary_logloss: 0.564908	valid_1's binary_logloss: 0.590454
[10]	valid_0's binary_logloss: 0.561782	valid_1's binary_logloss: 0.588422
[11]	valid_0's binary_logloss: 0.558941	valid_1's binary_logloss: 0.586723
[12]	valid_0's binary_logloss: 0.556497	valid_1's binary_logloss: 0.585544
[13]	valid_0's binary_logloss: 0.554298	valid_1's binary_logloss: 0.584415
[14]	valid_0's binary_logloss: 0.552399	valid_1's binary_logloss: 0.583648
[15]	valid_0's binary_logloss: 0.550605	valid_1's binary_logloss: 0.582899
[16]	valid_0's binary_logloss: 0.549085	valid_1's binary_logloss: 0.582321
[17]	valid_0's binary_logloss: 0.547605	valid_1's binary_logloss: 0.581557
[18]	valid_0's binary_logloss: 0.546426	valid_1's binary_logloss: 0.581518
[19]	valid_0's binary_logloss: 0.545225	valid_1's binary_logloss: 0.581406
[20]	valid_0's binary_logloss: 0.544178	valid_1's binary_logloss: 0.581194
[21]	valid_0's binary_loglo

...

[51]	valid_0's binary_logloss: 0.497443	valid_1's binary_logloss: 0.584836
[52]	valid_0's binary_logloss: 0.496446	valid_1's binary_logloss: 0.584817
[53]	valid_0's binary_logloss: 0.495242	valid_1's binary_logloss: 0.584947
[1]	valid_0's binary_logloss: 0.62227	valid_1's binary_logloss: 0.633223
[2]	valid_0's binary_logloss: 0.608788	valid_1's binary_logloss: 0.623159
[3]	valid_0's binary_logloss: 0.597853	valid_1's binary_logloss: 0.615108
[4]	valid_0's binary_logloss: 0.588558	valid_1's binary_logloss: 0.608671
[5]	valid_0's binary_logloss: 0.58076	valid_1's binary_logloss: 0.603433
[6]	valid_0's binary_logloss: 0.574143	valid_1's binary_logloss: 0.598798
[7]	valid_0's binary_logloss: 0.568447	valid_1's binary_logloss: 0.595341
[8]	valid_0's binary_logloss: 0.563449	valid_1's binary_logloss: 0.592556
[9]	valid_0's binary_logloss: 0.559223	valid_1's binary_logloss: 0.59038
[10]	valid_0's binary_logloss: 0.555242	valid_1's binary_logloss: 0.588713
[11]	valid_0's binary_logloss: 0.5518

[52]	valid_0's binary_logloss: 0.495221	valid_1's binary_logloss: 0.585641
[1]	valid_0's binary_logloss: 0.622371	valid_1's binary_logloss: 0.63327
[2]	valid_0's binary_logloss: 0.608883	valid_1's binary_logloss: 0.622702
[3]	valid_0's binary_logloss: 0.597831	valid_1's binary_logloss: 0.614471
[4]	valid_0's binary_logloss: 0.588532	valid_1's binary_logloss: 0.607919
[5]	valid_0's binary_logloss: 0.58072	valid_1's binary_logloss: 0.602916
[6]	valid_0's binary_logloss: 0.574113	valid_1's binary_logloss: 0.598797
[7]	valid_0's binary_logloss: 0.568517	valid_1's binary_logloss: 0.595711
[8]	valid_0's binary_logloss: 0.563492	valid_1's binary_logloss: 0.592972
[9]	valid_0's binary_logloss: 0.559107	valid_1's binary_logloss: 0.590618
[10]	valid_0's binary_logloss: 0.555268	valid_1's binary_logloss: 0.588344
[11]	valid_0's binary_logloss: 0.551852	valid_1's binary_logloss: 0.586757
[12]	valid_0's binary_logloss: 0.548859	valid_1's binary_logloss: 0.585716
[13]	valid_0's binary_logloss: 0.546

[1]	valid_0's binary_logloss: 0.622082	valid_1's binary_logloss: 0.633136
[2]	valid_0's binary_logloss: 0.608587	valid_1's binary_logloss: 0.622757
[3]	valid_0's binary_logloss: 0.597432	valid_1's binary_logloss: 0.614123
[4]	valid_0's binary_logloss: 0.587876	valid_1's binary_logloss: 0.607549
[5]	valid_0's binary_logloss: 0.57986	valid_1's binary_logloss: 0.602709
[6]	valid_0's binary_logloss: 0.573016	valid_1's binary_logloss: 0.598803
[7]	valid_0's binary_logloss: 0.567216	valid_1's binary_logloss: 0.595234
[8]	valid_0's binary_logloss: 0.56234	valid_1's binary_logloss: 0.592506
[9]	valid_0's binary_logloss: 0.558239	valid_1's binary_logloss: 0.590568
[10]	valid_0's binary_logloss: 0.554453	valid_1's binary_logloss: 0.588717
[11]	valid_0's binary_logloss: 0.551207	valid_1's binary_logloss: 0.587142
[12]	valid_0's binary_logloss: 0.548311	valid_1's binary_logloss: 0.586181
[13]	valid_0's binary_logloss: 0.545667	valid_1's binary_logloss: 0.585532
[14]	valid_0's binary_logloss: 0.543

[29]	valid_0's binary_logloss: 0.5217	valid_1's binary_logloss: 0.583924
[30]	valid_0's binary_logloss: 0.520455	valid_1's binary_logloss: 0.584074
[31]	valid_0's binary_logloss: 0.519333	valid_1's binary_logloss: 0.584826
[32]	valid_0's binary_logloss: 0.51819	valid_1's binary_logloss: 0.585064
[33]	valid_0's binary_logloss: 0.517074	valid_1's binary_logloss: 0.585414
[34]	valid_0's binary_logloss: 0.516022	valid_1's binary_logloss: 0.58558
[35]	valid_0's binary_logloss: 0.514963	valid_1's binary_logloss: 0.585932
[36]	valid_0's binary_logloss: 0.513814	valid_1's binary_logloss: 0.586048
[37]	valid_0's binary_logloss: 0.512801	valid_1's binary_logloss: 0.586533
[38]	valid_0's binary_logloss: 0.511654	valid_1's binary_logloss: 0.586355
[39]	valid_0's binary_logloss: 0.510544	valid_1's binary_logloss: 0.586499
[40]	valid_0's binary_logloss: 0.509398	valid_1's binary_logloss: 0.586421
[41]	valid_0's binary_logloss: 0.508258	valid_1's binary_logloss: 0.586438
[42]	valid_0's binary_logloss

[1]	valid_0's binary_logloss: 0.622082	valid_1's binary_logloss: 0.633136
[2]	valid_0's binary_logloss: 0.608587	valid_1's binary_logloss: 0.622757
[3]	valid_0's binary_logloss: 0.597432	valid_1's binary_logloss: 0.614123
[4]	valid_0's binary_logloss: 0.587876	valid_1's binary_logloss: 0.607549
[5]	valid_0's binary_logloss: 0.57986	valid_1's binary_logloss: 0.602709
[6]	valid_0's binary_logloss: 0.573016	valid_1's binary_logloss: 0.598803
[7]	valid_0's binary_logloss: 0.567216	valid_1's binary_logloss: 0.595234
[8]	valid_0's binary_logloss: 0.56234	valid_1's binary_logloss: 0.592506
[9]	valid_0's binary_logloss: 0.558239	valid_1's binary_logloss: 0.590568
[10]	valid_0's binary_logloss: 0.554453	valid_1's binary_logloss: 0.588717
[11]	valid_0's binary_logloss: 0.551207	valid_1's binary_logloss: 0.587142
[12]	valid_0's binary_logloss: 0.548311	valid_1's binary_logloss: 0.586181
[13]	valid_0's binary_logloss: 0.545667	valid_1's binary_logloss: 0.585532
[14]	valid_0's binary_logloss: 0.543

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

def get_clf_eval(y_test , pred, pred_proba):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [21]:
from lightgbm import LGBMClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


x_features = final13.drop(['CUSTNO','target_grade'], axis=1, inplace=False)
y_target = final13['target_grade']
# 전체 데이터 중, 80%는 학습용 데이터, 20%는 테스트용 데이터 추출
x_train, x_test, y_train, y_test = train_test_split(x_features, y_target, test_size = 0.2, random_state=156)

# 위에서 만든 x_train, y_train을 다시 쪼개서 90%는 학습과 10%는 검증용 데이터로 분리
x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=156)

# 앞서 XGBoost와 동일하게 n_estimators는 400 설정
lgbm_wrapper = LGBMClassifier(n_estimators=400, learning_rate=0.05)

#lightGBM도 XGBoost와 동일하게 조기 중단 수행 가능
evals = [(x_tr,y_tr),(x_val,y_val)]
lgbm_wrapper.fit(x_tr,y_tr, early_stopping_rounds=50, eval_metric='logloss', eval_set=evals)
pred = lgbm_wrapper.predict(x_test)
pred_proba = lgbm_wrapper.predict_proba(x_test)[:,1]

get_clf_eval(y_test , pred, pred_proba)

[1]	training's binary_logloss: 0.631305	valid_1's binary_logloss: 0.64403
[2]	training's binary_logloss: 0.624138	valid_1's binary_logloss: 0.637168
[3]	training's binary_logloss: 0.617617	valid_1's binary_logloss: 0.631219
[4]	training's binary_logloss: 0.611726	valid_1's binary_logloss: 0.625727
[5]	training's binary_logloss: 0.606372	valid_1's binary_logloss: 0.621124
[6]	training's binary_logloss: 0.601463	valid_1's binary_logloss: 0.616866
[7]	training's binary_logloss: 0.596998	valid_1's binary_logloss: 0.613092
[8]	training's binary_logloss: 0.5929	valid_1's binary_logloss: 0.609229
[9]	training's binary_logloss: 0.589078	valid_1's binary_logloss: 0.605896
[10]	training's binary_logloss: 0.585612	valid_1's binary_logloss: 0.602984
[11]	training's binary_logloss: 0.582323	valid_1's binary_logloss: 0.600172
[12]	training's binary_logloss: 0.579408	valid_1's binary_logloss: 0.597788
[13]	training's binary_logloss: 0.576665	valid_1's binary_logloss: 0.595557
[14]	training's binary_l

오차 행렬
[[2053  372]
 [ 676  560]]
정확도: 0.7137, 정밀도: 0.6009, 재현율: 0.4531,    F1: 0.5166, AUC:0.7260


In [16]:
from lazypredict.Supervised import LazyClassifier, LazyRegressor

x_features = final13.drop(['CUSTNO','target_grade'], axis=1, inplace=False)
y_target = final13['target_grade']

x_train, x_test, y_train, y_test = train_test_split(x_features, y_target, test_size = 0.2, random_state=156)

clf = LazyClassifier(verbose=0, predictions=True)
# clf2 = LazyRegressor(verbose=0, predictions=True)

# models, predictions = clf2.fit(x_train, x_test, y_train, y_test)
models, predictions = clf.fit(x_train, x_test, y_train, y_test)
models

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [01:31<00:00,  3.15s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.71,0.64,0.64,0.70,0.27
AdaBoostClassifier,0.70,0.64,0.64,0.69,0.52
RandomForestClassifier,0.70,0.64,0.64,0.69,1.70
XGBClassifier,0.69,0.63,0.63,0.68,0.65
ExtraTreesClassifier,0.70,0.63,0.63,0.69,1.82
LinearDiscriminantAnalysis,0.69,0.62,0.62,0.67,0.09
QuadraticDiscriminantAnalysis,0.69,0.62,0.62,0.67,0.04
CalibratedClassifierCV,0.69,0.61,0.61,0.67,5.46
LogisticRegression,0.69,0.61,0.61,0.67,0.06
